# 데이터프레임 전처리

In [19]:
import json
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [185]:
# 데이터 불러오기
MP = pd.read_json('MP.json', orient = 'table')
PB = pd.read_json("PB.json", orient = 'table')

In [215]:
# 국회의원 별 df
MP.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 298
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   의원명     299 non-null    object
 1   정당      299 non-null    object
 2   소속위원회   298 non-null    object
 3   지역      299 non-null    object
 4   성별      299 non-null    object
 5   당선횟수    299 non-null    object
 6   당선방법    299 non-null    object
dtypes: object(7)
memory usage: 18.7+ KB


In [216]:
# 법률안 별 df
PB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1766 entries, 0 to 1765
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   법률안명    1766 non-null   object        
 1   대표발의자   1766 non-null   object        
 2   공동발의자   1758 non-null   object        
 3   소관위원회   1766 non-null   object        
 4   제안일     1766 non-null   datetime64[ns]
 5   처리상태    71 non-null     object        
 6   상세페이지   1766 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 174.9+ KB


In [217]:
# 의원별로 새로운 df 생성
person_li = MP['의원명'].tolist() + MP['의원명'].tolist()
person_se = pd.Series(person_li)

name_pb = pd.DataFrame(person_se, columns = ['의원명'])

#이름 오름차순 정렬
name_pb = name_pb.sort_values(['의원명'])

#인덱스 다시 설정
name_pb.reset_index(inplace=True, drop=True)

name_pb

,의원명
0,강기윤
1,강기윤
2,강대식
3,강대식
4,강득구
...,...
593,황보승희
594,황운하
595,황운하
596,황희


In [189]:
type(name_pb)

pandas.core.frame.DataFrame

In [218]:
# 발의자 설정(대표/ 공동)
name_pb['발의자'] = '대표'

In [219]:
name_pb

,의원명,발의자
0,강기윤,대표
1,강기윤,대표
2,강대식,대표
3,강대식,대표
4,강득구,대표
...,...,...
593,황보승희,대표
594,황운하,대표
595,황운하,대표
596,황희,대표


In [221]:
name_pb.iloc[1::2,1] = '공동'

In [222]:
name_pb

,의원명,발의자
0,강기윤,대표
1,강기윤,공동
2,강대식,대표
3,강대식,공동
4,강득구,대표
...,...,...
593,황보승희,공동
594,황운하,대표
595,황운하,공동
596,황희,대표


# 빈칸 만들기 -> df1

In [223]:
df1 = name_pb.copy()

In [224]:
df1.head()

,의원명,발의자
0,강기윤,대표
1,강기윤,공동
2,강대식,대표
3,강대식,공동
4,강득구,대표


In [282]:
import numpy as np

In [283]:
li_el = []
for _ in range(598): li_el.append([])

In [285]:
df1['법률안명'] = pd.Series(li_el)
df1['소관위원회'] = pd.Series(li_el)

In [286]:
df1.head()

,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,[],[]
1,강기윤,공동,[],[]
2,강대식,대표,[],[]
3,강대식,공동,[],[]
4,강득구,대표,[],[]


In [287]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   의원명     598 non-null    object
 1   발의자     598 non-null    object
 2   법률안명    598 non-null    object
 3   소관위원회   598 non-null    object
dtypes: object(4)
memory usage: 18.8+ KB


# ===== 여기서부터 ===== #

# 대표발의자에 대해 for문 돌릴때

## for문이 처음 돌아갈때

In [288]:
PB.head(2)

,법률안명,대표발의자,공동발의자,소관위원회,제안일,처리상태,상세페이지
0,성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김상훈, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, 조수진, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1,남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김승수, 배준영, 유경준, 유의동, 이명수, 정희용, 조수진, 지성호]",환경노동위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...


In [289]:
df2 = df1.copy()
# for i, ps in enumerate(PB['대표발의자']):
# for문이 처음 돌아가는 상황 재현
i = 0
ps = '강대식' 

idx = df2[(ps == df2['의원명'])&(df2['발의자'] == '대표')].index

print(df2[(ps == df2['의원명'])&(df2['발의자'] == '대표')])
print('*'*30)
print(idx.values)
print('*'*30)

old = list(df2.loc[idx,'법률안명'])
add = PB.loc[i, '법률안명']
new = [old[0] + [add]]
print(new)
print('*'*30)

df2.loc[idx,'법률안명'] = new
print(df2.loc[idx,:])

df2.head()

   의원명 발의자 법률안명 소관위원회
2  강대식  대표   []    []
******************************
[2]
******************************
[['성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안']]
******************************
   의원명 발의자                                법률안명 소관위원회
2  강대식  대표  [성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안]    []


,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,[],[]
1,강기윤,공동,[],[]
2,강대식,대표,[성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안],[]
3,강대식,공동,[],[]
4,강득구,대표,[],[]


## for문 두번째 돌아갈때

In [312]:
df3 = df2.copy()
# for i, ps in enumerate(PB['대표발의자']):
# for문이 두번째 돌아가는 상황 재현
i = 1
ps = '강대식' 

idx = df3[(ps == df3['의원명'])&(df3['발의자'] == '대표')].index

print(df3[(ps == df3['의원명'])&(df3['발의자'] == '대표')])
print('*'*30)
print(idx.values)
print('*'*30)

old = list(df3.loc[idx,'법률안명'])
add = PB.loc[i, '법률안명']
new = [old[0] + [add]]

print(new)
print('*'*30)

df3.loc[idx,'법률안명'] = new
print(df3.loc[idx,:])

df3.head()

   의원명 발의자                                법률안명 소관위원회
2  강대식  대표  [성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안]    []
******************************
[2]
******************************
[['성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안', '남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안']]
******************************
   의원명 발의자                                               법률안명 소관위원회
2  강대식  대표  [성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...    []


,의원명,발의자,법률안명,소관위원회
0,강기윤,대표,[],[]
1,강기윤,공동,[],[]
2,강대식,대표,"[성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...",[]
3,강대식,공동,[],[]
4,강득구,대표,[],[]


## 값이 잘 들어갔는지 확인

In [300]:
temp = df3.loc[idx,'법률안명']

In [311]:
print(type(temp), '\n', temp)
print(temp[2])

<class 'pandas.core.series.Series'> 
 2    [성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안, 남녀고용평등과 일·가...
Name: 법률안명, dtype: object
['성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안', '남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안']


# ===== 여기까지 ==== #

In [296]:
# 발의자가 대표인 경우
for i in list(PB['대표발의자']): #법률안 df 에서 의원명
    
    li = name_pb[(name_pb['의원명']==i)&(name_pb['발의자'] == '대표')]['법률안명'].tolist() #리스트로 변경
    j = name_pb.index[(name_pb['의원명']==i)&(name_pb['발의자'] == '대표')]
    j = j.tolist() #int64타입이라 리스트로 변경
    
    for t in list(PB[PB['대표발의자']==i]['법률안명']): #해당 법률안 리스트로 append
        li.append(t)
        
    name_pb.iloc[j[0],2] = str(li) #li 값 추가
        
name_pb

KeyError: '법률안명'

In [ ]:
li = name_pb[(name_pb['의원명']=='강대식')&(name_pb['발의자'] == '대표')]['법률안명'].tolist() 

In [ ]:
j = name_pb.index[(name_pb['의원명']=='강대식')&(name_pb['발의자'] == '대표')]

In [ ]:
j = j.tolist()

In [ ]:
for i in list(PB[PB['대표발의자']=='강대식']['법률안명']):
    li.append(i)
    
li

In [ ]:
name_pb.iloc[j[0],2] = str(li)

In [ ]:
name_pb.iloc[j[0],2]

In [ ]:
name_pb.iloc[j[0],:]